<a href="https://colab.research.google.com/github/ayushved78/MLE/blob/main/Linear%20Regression/Ridge_Lasso_ElasticNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### All Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')

### Import Dataset

In [ ]:
import kagglehub
path = kagglehub.dataset_download("nitinchoudhary012/algerian-forest-fires-dataset")
print("Path to dataset files:", path)
try:
    files = os.listdir(path)
    for file in files:
        print(file)
except FileNotFoundError:
    print(f"Directory not found: {path}")

In [ ]:
df = pd.read_csv(path + "/Algerian_forest_fires_dataset.csv")
df.head()

### Data Cleaning

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.loc[:122,"Region"]=0
df.loc[122:,"Region"]=1

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.iloc[122]
# remove 122
df = df.drop(122).reset_index(drop=True)

In [ ]:
df.iloc[122]

### Changing Column dtypes

In [ ]:
df.columns

In [ ]:
# fix spaces in column names
df.columns = df.columns.str.strip()
df.columns

In [ ]:
df[['day', 'month', 'year', 'Temperature', 'RH', 'Ws']] = df[['day', 'month', 'year', 'Temperature', 'RH', 'Ws']].astype(int)

In [ ]:
objects = [features for features in df.columns if df[features].dtypes=='O']

In [ ]:
for i in objects:
  if i!='Classes':
    df[i]=df[i].astype(float)

In [ ]:
df[['Region']]=df[['Region']].astype(int)

In [ ]:
df.info()

In [ ]:
df.to_csv('Algerian_forest_fires_cleaned.csv', index=False)

### EDA

In [ ]:
# drop day, month, year
df1 = df.drop(['day','month','year'],axis=1)
df1.head()

In [ ]:
df1['Classes'].value_counts()

In [ ]:
df1['Classes'] = np.where(df1['Classes'].str.contains('not'), 0, 1)

In [ ]:
df1.head()

In [ ]:
plt.style.use('fivethirtyeight')
df1.hist(figsize=(15,15))
plt.show()

In [ ]:
class_counts = df1['Classes'].value_counts()
labels = ['Not Fire', 'Fire']  # Corrected labels
sizes = [class_counts[0], class_counts[1]]

plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Fire vs Not Fire')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
df1.corr()

In [ ]:
sns.heatmap(df1.corr())

In [ ]:
df['Classes'] = np.where(df['Classes'].str.contains('not'), 0, 1)

In [ ]:
df['Classes'].value_counts()

### Independent and Dependent Features

In [ ]:
X = df1.drop('FWI',axis=1)
Y = df1['FWI']

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(X_train.corr(),annot=True)

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_set = correlation(X_train,0.85)

In [ ]:
X_train.drop(corr_set,axis=1,inplace=True)
X_test.drop(corr_set,axis=1,inplace=True)
X_train.shape,X_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)